In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

# Layers Imports
from tensorflow.keras.layers import Dense , InputLayer , Conv2D , MaxPool2D , Flatten , BatchNormalization
from tensorflow.keras.models import Sequential

# Compiling imports

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy


In [3]:
dataset , dataset_info = tfds.load('malaria' , with_info=True , as_supervised = True , shuffle_files = True , split=['train'])
dataset_info

tfds.core.DatasetInfo(
    name='malaria',
    full_name='malaria/1.0.0',
    description="""
    The Malaria dataset contains a total of 27,558 cell images with equal instances
    of parasitized and uninfected cells from the thin blood smear slide images of
    segmented cells.
    """,
    homepage='https://lhncbc.nlm.nih.gov/publication/pub9932',
    data_dir='C:\\Users\\SHUBHAM RAJ\\tensorflow_datasets\\malaria\\1.0.0',
    file_format=tfrecord,
    download_size=337.08 MiB,
    dataset_size=317.62 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=27558, num_shards=4>,
    },
    citation="""@article{rajaraman2018pre,
      title={Pre-trained convolutional neural networks as feature extractors toward
      improved malaria parasite detection in

In [4]:
def splits(dataset , TRAIN_RATIO , VAL_RATIO , TEST_RATIO):
    DATASET_SIZE = len(dataset)

    train_dataset = dataset.take(int(TRAIN_RATIO * DATASET_SIZE))

    val_dataset = dataset.skip(int(TRAIN_RATIO * DATASET_SIZE))
    val_dataset = dataset.take(int(VAL_RATIO * DATASET_SIZE))

    test_dataset = dataset.skip(int(TRAIN_RATIO * DATASET_SIZE))
    test_dataset = dataset.skip(int(VAL_RATIO * DATASET_SIZE))
    test_dataset = dataset.take(int(TEST_RATIO * DATASET_SIZE))

    return train_dataset , val_dataset , test_dataset


train_dataset , val_dataset , test_dataset = splits(dataset[0] , 0.6 , 0.2 , 0.2)

DATA PREPROCESSING

In [5]:
IMG_SIZE = 224
def resize_rescale(image , label):
    return tf.image.resize(image , (IMG_SIZE,IMG_SIZE))/255.0 , label

train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

train_dataset = train_dataset.shuffle(buffer_size = 8 , reshuffle_each_iteration = True ).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size = 8 , reshuffle_each_iteration = True ).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(1)

 MODEL BUILDING

In [6]:
model = Sequential([

    InputLayer(input_shape=(IMG_SIZE,IMG_SIZE,3)),

    Conv2D(filters=6 , kernel_size=5 , strides=1 ,padding ='valid' , activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2 , strides =2 ),

    Conv2D(filters=16 , kernel_size=5 , strides=1 ,padding ='valid' , activation='relu'),
    MaxPool2D(pool_size=2 , strides =2 ),

    Flatten(),

    Dense(100 , activation='relu'),
    BatchNormalization(),
    Dense(10 , activation='relu'),
    Dense(1 , activation='sigmoid')

])

EVALUATION

In [7]:
model.compile(optimizer=Adam(learning_rate=0.01) , loss = BinaryCrossentropy(), metrics='accuracy')

# Training

history =  model.fit(train_dataset , validation_data = val_dataset, epochs = 30, verbose = 1)

# Evaluation
model.evaluate(test_dataset)

Epoch 1/30
517/517 [==============================] - 786s 2s/step - loss: 0.5406 - accuracy: 0.7203 - val_loss: 1.1829 - val_accuracy: 0.4972
Epoch 2/30
517/517 [==============================] - 754s 1s/step - loss: 0.2608 - accuracy: 0.9056 - val_loss: 0.2172 - val_accuracy: 0.9388
Epoch 3/30
517/517 [==============================] - 1005s 2s/step - loss: 0.4399 - accuracy: 0.7825 - val_loss: 0.6256 - val_accuracy: 0.6427
Epoch 4/30
517/517 [==============================] - 862s 2s/step - loss: 0.5064 - accuracy: 0.7543 - val_loss: 2.1105 - val_accuracy: 0.4957
Epoch 5/30
517/517 [==============================] - 848s 2s/step - loss: 0.2341 - accuracy: 0.9165 - val_loss: 0.1877 - val_accuracy: 0.9307
Epoch 6/30
517/517 [==============================] - 833s 2s/step - loss: 0.1911 - accuracy: 0.9344 - val_loss: 0.4607 - val_accuracy: 0.9461
Epoch 7/30
517/517 [==============================] - 740s 1s/step - loss: 0.2015 - accuracy: 0.9298 - val_loss: 0.7127 - val_accuracy: 0.502

[0.044433921575546265, 0.981673002243042]

saving the model to deply on streamlit app

In [ ]:
model.save("malaria_detector.h5")